In [1]:
from openai import AzureOpenAI
from dotenv import dotenv_values
import json

In [2]:
from prompts import prompt_gpt

In [3]:
config =dotenv_values("Configuration.env")

In [4]:
endpoint=config['ENDPOINT_URL']
deployment=config['DEPLOYMENT_NAME']
search_endpoint=config['SEARCH_ENDPOINT']
search_key=config['SEARCH_KEY']
subscription_key=config['AZURE_OPENAI_API_KEY']

In [5]:
def invoke_llm(query):
    client =AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=subscription_key,
        api_version="2024-05-01-preview"
    )
    
    prompt=prompt_gpt.PROMPT
    messages = [
        {
            "role": "system",
            "content": f"{prompt}"
        },
        {
            "role":"user",
            "content":f"{query}"
        }        
    ]
    
    completion= client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=1000,
        temperature=0.1,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
        extra_body={
            "data_sources":[
                {
                    "type":"azure_search",
                    "parameters":{
                        "filter":None,
                        "endpoint": f"{search_endpoint}",
                        "index_name": config["INDEX_NAME"],
                        "semantic_configuration":"azureml-default",
                        "authentication": {
                            "type":"api_key",
                            "key":f"{search_key}"
                        },
                        "embedding_dependency":{
                            "type":"endpoint",
                            "endpoint":f"{endpoint}/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15",
                            "authentication":{
                                "type": "api_key",
                                "key":f"{subscription_key}"
                            }
                        },
                        "query_type":"vector_simple_hybrid",
                        "in_scope":True,
                        "role_information":f"{prompt}",
                        "strictness":3,
                        "top_n_documents":10
                    }
                }
            ]
        }
        
    )
    
    res=json.loads(completion.to_json())
    
    return res
    

In [ ]:
# query="How can I Update my Financial Institution Information?"
# query="What is a GCKey?"
query="I did not receive a welcome email. How do I register?"

In [7]:
result=invoke_llm(query=query)

In [8]:
print(result["choices"][0]["message"]['content'])


If you did not receive a welcome email, you can still register your NSLSC account by following these steps:

1. Go to the registration page and click "Click here to continue to register." You will be directed to enter your Date of Birth and Social Insurance Number (SIN) [doc3].
2. Generate your Secure Access Code (SAC). This code will be sent to the email address on file with your account. If you previously used your username and password, you will have the opportunity to update your email address at this step [doc3].
3. If you do not receive the Secure Access Code email within a few minutes, check your Spam and Junk folders or search your email for messages from "info@nslsc.ca" with the subject line "Your secure Access Code/Votre code d'accès sécurisé" [doc2].
4. Once you receive the Secure Access Code, enter it on the website and select "Continue." Keep this code handy, as it will be required again later in the registration process [doc2].

If you continue to experience issues receiv

In [21]:
from utils import get_logger,clean_html_tags

In [23]:
search_results=result["choices"][0]["message"]['context']['citations']


[({
    'raw_text': (clean_html_tags(doc['content'])[0:350] if "content" in doc else ""),
    'source_url': (doc['url'] if "url" in doc else "")
    })
for doc in search_results[:10]]

[{'raw_text': 'Title: How-to-Customize-My-Payment-Terms_E.pdf5.Review your revised repayment schedule. You can go back and make edits or select the Submit button to confirm changes. 6.A confirmation page will appear confirming the customization of your pay ment information. You will also receive an email confirming the changes. Page 3',
  'source_url': 'How-to-Customize-My-Payment-Terms_E.pdf'},
 {'raw_text': 'Title: How-to-Register-your-NSLSC-Account_Oct2023_EN.pdf Page 19 It can take a few minutes to receive the email containing your temporary Secure Access Code (SAC) . If you can’t find the email, please try the following: • Check your Spam and Junk folder • Search your email for messages from info@nslsc.ca with the subject line Your secure Access Cod',
  'source_url': 'How-to-Register-your-NSLSC-Account_Oct2023_EN.pdf'},
 {'raw_text': 'Title: How-to-Register-your-NSLSC-Account_Oct2023_EN.pdf Page 18 3c. Upon clicking Click here to continue to register , you will be directed to a ne

In [17]:
search_results=result["choices"][0]["message"]['context']['citations']

In [20]:
search_results[0]['content']

'Title: How-to-Customize-My-Payment-Terms_E.pdf5.Review your revised repayment schedule. You can go back and make edits or select the\nSubmit button to confirm changes.\n6.A confirmation\n page will appear confirming the customization of your pay ment information. You will\nalso receive an email confirming the changes.\nPage 3\n'

### enable streamng

In [6]:
query="How can I Update my Financial Institution Information?"
client =AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-05-01-preview"
)

prompt=prompt_gpt.PROMPT
messages = [
    {
        "role": "system",
        "content": f"{prompt}"
    },
    {
        "role":"user",
        "content":f"{query}"
    }        
]

completion= client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=1000,
    temperature=0.1,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=True,
    extra_body={
        "data_sources":[
            {
                "type":"azure_search",
                "parameters":{
                    "filter":None,
                    "endpoint": f"{search_endpoint}",
                    "index_name": config["INDEX_NAME"],
                    "semantic_configuration":"azureml-default",
                    "authentication": {
                        "type":"api_key",
                        "key":f"{search_key}"
                    },
                    "embedding_dependency":{
                        "type":"endpoint",
                        "endpoint":f"{endpoint}/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15",
                        "authentication":{
                            "type": "api_key",
                            "key":f"{subscription_key}"
                        }
                    },
                    "query_type":"vector_simple_hybrid",
                    "in_scope":True,
                    "role_information":f"{prompt}",
                    "strictness":3,
                    "top_n_documents":10
                }
            }
        ]
    }
    
)

for update in completion:
    if update.choices:
        print(update.choices[0].delta.content or "",end="")

client.close()

To update your Financial Institution Information with the National Student Loans Service Centre (NSLSC), follow these steps:

1. Log in to your NSLSC account [doc1].
2. Select "Financial Institution Information" from the "My Account" drop-down menu [doc1].
3. Add or update your financial institution information [doc2].
4. Enter your financial institution details, review and accept the Terms and Conditions, and click "Update" to complete the process [doc2].
5. After submission, you will be redirected to the Financial Institution Information page, where a successful update message will appear. Additionally, a confirmation message will be sent to your NSLSC inbox [doc3].